Display search notebook component?



In [ ]:
import { Component, ModuleWithProviders, NgModule } from '@angular/core';
import { COMMON_MODULES } from '../../../imports/core.module';
import { RouterModule, Routes } from '@angular/router';
import { SearchService } from '../../../imports/search.service';

@Component({
    selector: 'bc-search',
    template: `
        <form>
            <md-input-container>
                <input mdInput name="search" required type="text"
                       placeholder="Search"
                       maxlength="100" [(ngModel)]="query" (change)="search()">
            </md-input-container>
        </form>
    `,
    styles: [ `
        md-input-container {
            width: 100%;
        }
    ` ]
})
export class SearchComponent {
    query = '';

    constructor(public service: SearchService) {
    }

    search(): void {
        this.service.search(this.query).subscribe(r => {
            console.log('searched ' + r);
        });
    }
}

export const COMPONENTS = [
    SearchComponent
];

export const authRoutes: Routes = [
    {
        path: '',
        component: SearchComponent,
        data: {roles: [ 'anonymous', 'user' ]}
    }
];
export const routing: ModuleWithProviders = RouterModule.forChild(authRoutes);

@NgModule({
    imports: [
        ...COMMON_MODULES,
        routing
    ],
    declarations: COMPONENTS,
    exports: COMPONENTS
})
export class SearchModule {
}



Search notebook service?



In [ ]:
import { Observable } from 'rxjs/Observable';
import { Http } from '@angular/http';
import { Injectable } from '@angular/core';
import { Response } from '@angular/http';

export let callbackUrl = 'localhost';

@Injectable()
export class SearchService {
    constructor(public http: Http) {
    }

    search(query: string): Observable<Response> {
        console.log('Searching ' + query);
        return this.http.post(callbackUrl, {query});
    }
}


Display code results?

In [ ]:
import { SearchService } from '../../../imports/search.service';
import { ChangeDetectorRef, Component, OnDestroy, OnInit } from '@angular/core';
import { Subscription } from 'rxjs/Subscription';
import * as Prism from 'prismjs';

@Component({
    selector: 'bc-results',
    template: `
        <pre [class]="'language-'+(r.lang||'javascript')" *ngFor="let r of results" [innerHTML]="highlight(r.code)">
        </pre>
    `,
    styleUrls: [ './results.component.scss' ]
})
export class ResultsComponent implements OnInit, OnDestroy {
    query = '';
    results: Array<any> = [];
    private resultsSub: Subscription;

    constructor(public service: SearchService,
                public ref: ChangeDetectorRef) {
    }

    ngOnInit(): void {
        this.resultsSub = this.service.results(this.query).subscribe(r => {
            this.results = [ (r as Array<any>)[ 0 ] ];
            this.ref.detectChanges();
        });
    }

    ngOnDestroy(): void {
        this.resultsSub.unsubscribe();
    }

    highlight(code: string): string {
        return Prism.highlight(code, Prism.languages.javascript);
    }
}



Sockify search notebook provider?

In [1]:
var io = require('socket.io-client');
var importer = require('../Core');
var path = require('path');
var interpretAllPath = path.join(__dirname, './interpret all notebooks.ipynb');

var interpret, interpretObject;
var interpretAll = (queries, root = '') => {
    return importer.importNotebook(interpretAllPath)
    .then((f) => {
        interpret = f['interpret'];
        interpretObject = f['interpretObject'];
    })
    // use Promise.all to do all of your code searches up front
    .then(r => typeof queries == 'string' 
          ? interpret(queries) 
          : Promise.all(queries.map(interpret)))
    .then(r => interpretObject(r))
};

var searchHandler = () => {
    var client = io('https://localhost:8000', {
        secure: true,
        rejectUnauthorized: false 
    });
    return new Promise(resolve => {
        console.log('Search handler');
        client.on('resolve', (name, search) => {
            console.log('searching...');
            var interpretObject;
            if(name == 'SearchService.prototype.search') {
                client.emit('result', 'SearchService.prototype.search',
                            'Searching notebooks for ' + search);
                // TODO: include gulp notebook search, .bash_sessions,
                //    selenium scripts, stack overflow, github, 
                interpretAll(search)
                    .then(r => {
                        client.emit('result', 'SearchService.prototype.results', r);
                    });
            }
        });
        client.emit('handle', 'SearchService', () => resolve());
        client.on('error', e => console.log(e))
    })
};
module.exports = searchHandler;
searchHandler;



[Function: searchHandler]

In [ ]:
$$.async();
searchHandler()
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))

Search handler


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/universal';
var server = require(project + '/src/server/sockify-server.js').sockifyServer(8000);
var client = require('socket.io-client').connect('https://localhost:8000');
client.emit('close');


Display login form?



In [ ]:
import { RouterModule, Routes } from '@angular/router';
import { Component, ModuleWithProviders, NgModule } from '@angular/core';
import { COMMON_MODULES } from '../../../imports/core.module';
import { AuthService } from '../../../imports/auth.service';

@Component({
    selector: 'bc-login',
    template: `
        <form>
            <md-input-container>
                <input mdInput name="signupEmail" required type="email"
                       placeholder="Username"
                       maxlength="100" [(ngModel)]="username">
            </md-input-container>
            <br/>
            <md-input-container>
                <input mdInput name="signupPassword" required type="password" maxlength="25"
                       placeholder="Password"
                       length="40" [(ngModel)]="password">
            </md-input-container>
            <button md-raised-button color="primary"
                    id="signupSubmit" type="submit" (click)="onLogin()">
                Login
            </button>
        </form>
    `,
    styleUrls: [ './login.component.scss' ]
})
export class LoginComponent {
    username: string;
    password: string;

    constructor(public service: AuthService) {
    }

    onLogin(): void {
        this.service.login(this.username, this.password).subscribe(r => {
            console.log(r);
        });
    }
}

export const COMPONENTS = [
    LoginComponent
];

export const authRoutes: Routes = [
    {
        path: '',
        component: LoginComponent,
        data: {roles: [ 'anonymous', 'user' ]}
    }
];
export const routing: ModuleWithProviders = RouterModule.forChild(authRoutes);

@NgModule({
    imports: [
        ...COMMON_MODULES,
        routing
    ],
    declarations: COMPONENTS,
    exports: COMPONENTS
})
export class AuthModule {
}



Authentication service?
    


In [ ]:
import { Observable } from 'rxjs/Observable';
import { Http } from '@angular/http';
import { Injectable } from '@angular/core';

export let callbackUrl = 'localhost';

@Injectable()
export class AuthService {
    constructor(public http: Http) {
    }

    login(username: string, password: string): Observable<number> {
        return this.http.post(callbackUrl, {username, password})
            .map(r => r.json());
    }
}

